In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

In [ ]:
files.upload();
data = pd.read_csv("vgames2.csv", index_col=0)
data.head()

In [ ]:
'''
# Data Description

Name : 게임의 이름입니다.
Platform : 게임이 지원되는 플랫폼의 이름입니다.
Year : 게임이 출시된 연도입니다.
Genre : 게임의 장르입니다.
Publisher : 게임을 제작한 회사입니다.
NA_Sales : 북미지역에서의 출고량입니다.
EU_Sales : 유럽지역에서의 출고량입니다.
JP_Sales : 일본지역에서의 출고량입니다.
Other_Sales : 기타지역에서의 출고량입니다.


# Keywords (참조용)

Sprint 1 : EDA, Feature Engineering, Data Manipulation, Data Visualization
Sprint 2 : Hypothesis Test, CI, Bayesian, AB Test
Sprint 3 : DR, PCA, Clustering, GD


# 반드시 들어가야 하는 내용.

지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.
연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.
출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.
'''

In [ ]:
data.duplicated().sum() # 중복 데이터 확인

In [ ]:
data.isnull().sum() # 결측 데이터 확인

In [ ]:
data[data.Year.isnull()==True] # Name에 결측이 있는 것은 아니므로 검색을 통해 결측치를 채우되, 양이 많으므로 파일 자체에서 수정

In [ ]:
files.upload();

In [ ]:
data2 = pd.read_csv("vgames2_revised.csv", index_col=0)
data2.head()

In [ ]:
data2.isnull().sum()

In [ ]:
data2[data2.Other_Sales.isnull()==True] # Other_sales가 null인 데이터 확인

In [ ]:
data2.loc[7597, 'Other_Sales'] = '0' # 결측치들 0으로 대체
data2.loc[16192, 'Other_Sales'] = '0'

In [ ]:
data2.isnull().sum()

In [ ]:
data2[data2.duplicated()==True] # 중복데이터 확인

In [ ]:
data2[data2.Name=='Wii de Asobu: Metroid Prime'] 

In [ ]:
data2.drop(index=9185, inplace=True) # 중복된 데이터 삭제
data2.reset_index(drop=True, inplace=True)
data2

In [ ]:
data2.info() # year는 int로, sales 부분도 100을 곱해서 int로 바꿔야 데이터 관리하기 편함

In [ ]:
data2[data2.Year == 'Adventure'] # 데이터 두개가 한칸씩 밀린것을 확인

In [ ]:
data2.iloc[7596, 2:5] = data2.iloc[7596, 1:4]
data2.iloc[16190, 2:5] = data2.iloc[16190,1:4]
data2.loc[7596, 'Name'], data2.loc[7596, 'Platform'] = data2.loc[7596, 'Name'].split("??,")[0], data2.loc[7596, 'Name'].split("??,")[1]
data2.loc[16190, 'Name'], data2.loc[16190, 'Platform'] = data2.loc[16190, 'Name'].split("??,")[0], data2.loc[16190, 'Name'].split("??,")[1]
data2.loc[7596, 'Platform'] = data2.loc[7596, 'Platform'].replace('"', '')
data2.loc[16190, 'Platform'] = data2.loc[16190, 'Platform'].replace('"', '')

In [ ]:
data2.Year = data2.Year.astype(np.int32)

In [ ]:
data2[data2.Year < 1900] # Year에서 outlier 확인

In [ ]:
for i in range(len(data2)):
    if data2.iloc[i, 2] < 50: # data2.Year가 50 아래인 data는 2000년대로 간주하여 2000을,
        data2.iloc[i, 2] += 2000
    elif data2.iloc[i, 2] > 50 and data2.iloc[i, 2] < 100: # data2.Year가 50~100인 data는 1900년대로 간주하여 1900을 더함
        data2.iloc[i, 2] += 1900
data2.Year.describe()

In [ ]:
data2.info()

In [ ]:
data2[data2.NA_Sales.str.contains('[a-zA-Z]', regex=True)] # Sales에 문자 들어간 것들 확인

In [ ]:
NA_K_index = data2[data2.NA_Sales.str.contains(pat=r'K', regex=True)].index # 단위가 K인 index들 저장
EU_K_index = data2[data2.EU_Sales.str.contains(pat=r'K', regex=True)].index
JP_K_index = data2[data2.JP_Sales.str.contains(pat=r'K', regex=True)].index
Other_K_index = data2[data2.Other_Sales.str.contains(pat=r'K', regex=True, na=False)].index

In [ ]:
data2["NA_Sales"] = data2["NA_Sales"].str.replace(pat=r'[a-zA-Z]', repl=r'', regex=True)
data2.NA_Sales = data2.NA_Sales.astype(float)
data2["EU_Sales"] = data2["EU_Sales"].str.replace(pat=r'[a-zA-Z]', repl=r'', regex=True)
data2.EU_Sales = data2.EU_Sales.astype(float)
data2["JP_Sales"] = data2["JP_Sales"].str.replace(pat=r'[a-zA-Z]', repl=r'', regex=True)
data2.JP_Sales = data2.JP_Sales.astype(float)
data2["Other_Sales"] = data2["Other_Sales"].str.replace(pat=r'[a-zA-Z]', repl=r'', regex=True)
data2.Other_Sales = data2.Other_Sales.astype(float)

In [ ]:
data2.iloc[NA_K_index, 5] = data2.iloc[NA_K_index, 5] / 1000
data2.iloc[EU_K_index, 6] = data2.iloc[EU_K_index, 6] / 1000
data2.iloc[JP_K_index, 7] = data2.iloc[JP_K_index, 7] / 1000
data2.iloc[Other_K_index, 8] = data2.iloc[Other_K_index, 8] / 1000

In [ ]:
data2.NA_Sales = data2.NA_Sales * 100 # 소수점 두자리로 되어있는 데이터를 정수형으로 바꾸기 위해 100을 곱함
data2.NA_Sales = data2.NA_Sales.astype(int)
data2.EU_Sales = data2.EU_Sales * 100
data2.EU_Sales = data2.EU_Sales.astype(int)
data2.JP_Sales = data2.JP_Sales * 100
data2.JP_Sales = data2.JP_Sales.astype(int)
data2.Other_Sales = data2.Other_Sales * 100
data2.Other_Sales = data2.Other_Sales.astype(int)

In [ ]:
data2.describe() # sales의 단위는 0.01M=10K

In [ ]:
# 각 지역별 sales를 합한 총합 sales column 만들기
data2['Total_Sales'] = data2['NA_Sales'] + data2['EU_Sales'] + data2['JP_Sales'] + data2['Other_Sales']
data2.head()

In [ ]:
df = data2.copy()

In [ ]:
# 장르별 지역별 선호도 조사

print(f"NA/Total = {df.NA_Sales.sum()/df.Total_Sales.sum()}, \nEU/Total = {df.EU_Sales.sum()/df.Total_Sales.sum()}, \nJP/Total: {df.JP_Sales.sum()/df.Total_Sales.sum()}, \nOther/Total: {df.Other_Sales.sum()/df.Total_Sales.sum()}")

# 전체 출고량 중 NA가 49%, EU가 27%, JP가 14%, 나머지 지역이 9%를 차지함
# 어떤 장르에 대해 이 수치보다 크면 그 지역은 해당 장르를 선호한다고 볼 수 있음

In [ ]:
df.groupby('Genre').count()

In [ ]:
genres = df['Genre'].unique()
platforms = df["Platform"].unique()
NA_genres_pop = []
for genre in genres:
    print(f"{genre}: {df[df.Genre==genre].NA_Sales.sum()/df[df.Genre==genre].Total_Sales.sum()}")
    NA_genres_pop.append(df[df.Genre==genre].NA_Sales.sum()/df[df.Genre==genre].Total_Sales.sum())

plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=NA_genres_pop)
plt.axhline(df.NA_Sales.sum()/df.Total_Sales.sum(), label='Mean of Popularity')
plt.xticks(rotation=45)
plt.ylim(0.3, 0.6)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in NA')
plt.legend()

In [ ]:
EU_genres_pop = []
for genre in genres:
    print(f"{genre}: {df[df.Genre==genre].EU_Sales.sum()/df[df.Genre==genre].Total_Sales.sum()}")
    EU_genres_pop.append(df[df.Genre==genre].EU_Sales.sum()/df[df.Genre==genre].Total_Sales.sum())

plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=EU_genres_pop)
plt.axhline(df.EU_Sales.sum()/df.Total_Sales.sum(), label='Mean of Popularity')
plt.xticks(rotation=45)
plt.ylim(0.15, 0.35)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in EU')
plt.legend();

In [ ]:
JP_genres_pop = []
for genre in genres:
    print(f"{genre}: {df[df.Genre==genre].JP_Sales.sum()/df[df.Genre==genre].Total_Sales.sum()}")
    JP_genres_pop.append(df[df.Genre==genre].JP_Sales.sum()/df[df.Genre==genre].Total_Sales.sum())

plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=JP_genres_pop)
plt.axhline(df.JP_Sales.sum()/df.Total_Sales.sum(), label='Mean of Popularity')
plt.xticks(rotation=45)
plt.ylim(0, 0.4)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in JP')
plt.legend();

NA 지역에선 Shooter와 Platform 장르가,
EU 지역에선 Action, Racing, Shooter 장르가,
JP 지역에선 Role-Playing과 Strategy 장르가 우세함을 볼 수 있음

반대로 NA 지역에선 Role-Playing과 Stratege 장르가,
EU 지역에선 Role-Playing, Puzzle, Fighting 장르가,
JP 지역에선 Action, Racing, Shooter 장르가 고전을 면치 못함

NA와 EU 지역은 혼자서 하는 장르를 선호하고 JP 지역은 불특정 다수와 함께하는 장르를 선호한다고 볼 수 있는데,
이는 NA, EU 지역은 개인주의에서, JP는 집단주의에서 영향을 받았다고 생각할 수 있음

In [ ]:
# 연도별 트렌드 분석
# ~1990, 1991~2000, 2001~2010, 2011~2020으로 연도 나누기
year_1990 = df.query("Year<=1990")
year_2000 = df.query("1990<Year<=2000")
year_2010 = df.query("2000<Year<=2010")
year_2020 = df.query("2010<Year<=2020")

In [ ]:
year_1990_genre_pop = []
for genre in genres:
    year_1990_genre_pop.append(year_1990[year_1990.Genre==genre].Total_Sales.sum()/year_1990.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=year_1990_genre_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.4)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 1980s')
plt.legend();

In [ ]:
year_2000_genre_pop = []
for genre in genres:
    year_2000_genre_pop.append(year_2000[year_2000.Genre==genre].Total_Sales.sum()/year_2000.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=year_2000_genre_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.4)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 1990s')
plt.legend();

In [ ]:
year_2010_genre_pop = []
for genre in genres:
    year_2010_genre_pop.append(year_2010[year_2010.Genre==genre].Total_Sales.sum()/year_2010.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=year_2010_genre_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.4)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 2000s')
plt.legend();

In [ ]:
year_2020_genre_pop = []
for genre in genres:
    year_2020_genre_pop.append(year_2020[year_2020.Genre==genre].Total_Sales.sum()/year_2020.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=genres, y=year_2020_genre_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.4)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 2010s')
plt.legend();

1990년 이전까진 Platform 장르가 절대 강세였지만 점차 영향력이 줄었고

Action 장르의 인기는 꾸준히 증가하여 2010년대에는 가장 인기있는 장르가 됨

In [ ]:
# 각 시대별 어떤 platform의 게임들이 있기 많았는지 확인
year_1990_platform_pop = []
for platform in platforms:
    year_1990_platform_pop.append(year_1990[year_1990.Platform==platform].Total_Sales.sum()/year_1990.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=platforms, y=year_1990_platform_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.6)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 1980s')
plt.legend()

In [ ]:
year_2000_platform_pop = []
for platform in platforms:
    year_2000_platform_pop.append(year_2000[year_2000.Platform==platform].Total_Sales.sum()/year_2000.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=platforms, y=year_2000_platform_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.6)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 1990s')
plt.legend();

In [ ]:
year_2010_platform_pop = []
for platform in platforms:
    year_2010_platform_pop.append(year_2010[year_2010.Platform==platform].Total_Sales.sum()/year_2010.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=platforms, y=year_2010_platform_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.6)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 2000s')
plt.legend();

In [ ]:
year_2020_platform_pop = []
for platform in platforms:
    year_2020_platform_pop.append(year_2020[year_2020.Platform==platform].Total_Sales.sum()/year_2020.Total_Sales.sum())
plt.figure(figsize=(10, 5))
sns.barplot(x=platforms, y=year_2020_platform_pop)
plt.xticks(rotation=45)
plt.ylim(0, 0.6)
plt.ylabel('Relative Popularity')
plt.title('Relative Popularity by Genre in 2010s')
plt.legend();

# 같은 시리즈임에도 시대별로 넘버링이 달라 데이터가 해당 platform을 제대로 대변하지 못함

nintendo: ds, wii 3ds 64 wiiU

ps: ps ps2 ps3 ps4 psv

psp: psp

pc: pc

gb: gb gba gc

xb: xb x360 xone

atari: 2600

기타: ng 3do ws tg16 pcfx

패미컴: nes snes

sega: dc gen gg scd SAT

In [ ]:
# 넘버링만 다른, 같은 series의 platform들을 하나로 통합
df['Platform_new'] = 'a'
for i in range(len(df)):
    if df.loc[i, 'Platform'] == 'DS' or df.loc[i, 'Platform'] == 'Wii' or df.loc[i, 'Platform'] == 'N64' or df.loc[i, 'Platform'] == '3DS' or df.loc[i, 'Platform'] == 'WiiU':
        df.loc[i, 'Platform_new'] = 'Nintendo'
    elif df.loc[i, 'Platform'] == 'PS' or df.loc[i, 'Platform'] == 'PS2' or df.loc[i, 'Platform'] == 'PS3' or df.loc[i, 'Platform'] == 'PS4' or df.loc[i, 'Platform'] == 'PSV':
        df.loc[i, 'Platform_new'] = 'PlayStation'
    elif df.loc[i, 'Platform'] == 'PSP':
        df.loc[i, 'Platform_new'] = 'PSP'
    elif df.loc[i, 'Platform'] == 'PC':
        df.loc[i, 'Platform_new'] = 'PC'
    elif df.loc[i, 'Platform'] == 'GB' or df.loc[i, 'Platform'] == 'GBA' or df.loc[i, 'Platform'] == 'GC':
        df.loc[i, 'Platform_new'] = 'GameBoy'
    elif df.loc[i, 'Platform'] == 'XB' or df.loc[i, 'Platform'] == 'X360' or df.loc[i, 'Platform'] == 'XONE':
        df.loc[i, 'Platform_new'] = 'Xbox'
    elif df.loc[i, 'Platform'] == '2600':
        df.loc[i, 'Platform_new'] = 'Atari 2600'
    elif df.loc[i, 'Platform'] == 'NES' or df.loc[i, 'Platform'] == 'SNES':
        df.loc[i, 'Platform_new'] = 'Famicom'
    elif df.loc[i, 'Platform'] == 'DC' or df.loc[i, 'Platform'] == 'GEN' or df.loc[i, 'Platform'] == 'GG' or df.loc[i, 'Platform'] == 'SCD' or df.loc[i, 'Platform'] == 'SAT':
        df.loc[i, 'Platform_new']= 'SEGA'
    else:
        df.loc[i, 'Platform_new'] = 'MISC'

In [ ]:
df

In [ ]:
year_1990_new = df.query("Year<=1990")
year_2000_new = df.query("1990<Year<=2000")
year_2010_new = df.query("2000<Year<=2010")
year_2020_new = df.query("2010<Year<=2020")

In [ ]:
platforms_new = df.Platform_new.unique()
year_1990_platform_pop = []
platform_label = []
total_sales = year_1990_new.Total_Sales.sum()

for platform in platforms_new:
    platform_total_sales = year_1990_new[year_1990_new.Platform_new==platform].Total_Sales.sum()
    if platform_total_sales != 0:
        platform_label.append(platform)
        year_1990_platform_pop.append(platform_total_sales / total_sales)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
plt.title('Market Share of Each Platform in 1980s', fontsize=14)
pie = ax.pie(x=year_1990_platform_pop, 
        labeldistance=1.0,
        startangle=90,
        autopct=lambda p : '{:.2f}%'.format(p))
plt.legend(pie[0],platform_label, loc=1) ## 범례 표시
plt.show()

In [ ]:
year_2000_platform_pop = []
platform_label = []
total_sales = year_2000_new.Total_Sales.sum()

for platform in platforms_new:
    platform_total_sales = year_2000_new[year_2000_new.Platform_new==platform].Total_Sales.sum()
    if platform_total_sales != 0:
        platform_label.append(platform)
        year_2000_platform_pop.append(platform_total_sales / total_sales)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
plt.title('Market Share of Each Platform in 1990s', fontsize=14)
pie = ax.pie(x=year_2000_platform_pop, 
        labeldistance=1.0,
        startangle=90,
        autopct=lambda p : '{:.2f}%'.format(p))
plt.legend(pie[0],platform_label, loc=1) ## 범례 표시
plt.show()

In [ ]:
year_2010_platform_pop = []
platform_label = []
total_sales = year_2010_new.Total_Sales.sum()

for platform in platforms_new:
    platform_total_sales = year_2010_new[year_2010_new.Platform_new==platform].Total_Sales.sum()
    if platform_total_sales != 0:
        platform_label.append(platform)
        year_2010_platform_pop.append(platform_total_sales / total_sales)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
plt.title('Market Share of Each Platform in 2000s', fontsize=14)
pie = ax.pie(x=year_2010_platform_pop, 
        labeldistance=1.0,
        startangle=90,
        autopct=lambda p : '{:.2f}%'.format(p))
plt.legend(pie[0],platform_label, loc=1) ## 범례 표시
plt.show()

In [ ]:
year_2020_platform_pop = []
platform_label = []
total_sales = year_2020_new.Total_Sales.sum()

for platform in platforms_new:
    platform_total_sales = year_2020_new[year_2020_new.Platform_new==platform].Total_Sales.sum()
    if platform_total_sales != 0:
        platform_label.append(platform)
        year_2020_platform_pop.append(platform_total_sales / total_sales)

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
plt.title('Market Share of Each Platform in 2010s', fontsize=14)
pie = ax.pie(x=year_2020_platform_pop, 
        labeldistance=1.0,
        startangle=90,
        autopct=lambda p : '{:.2f}%'.format(p))
plt.legend(pie[0],platform_label, loc=1) ## 범례 표시
plt.show()

1980년대에는 Nintendo 사의 Famicom과 GameBoy, Atari 사의 2600이 거의 대부분의 platform을 형성하였음

하지만 1990년대 출시한 Sony 사의 PlayStation이 엄청나게 빠르게 성장하며 50%에 달하는 점유율을 차지하였고

2000년대 이후에는 Nintendo 기기의 성장과 Xbox라는 경쟁상대의 등장으로 그 점유율이 줄었지만 여전히 40%에 가까운 점유율을 차지함

In [ ]:
# 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

df.Total_Sales.describe()

In [ ]:
sns.boxplot(x=df.Total_Sales)

In [ ]:
 # 총 출고량 상위 100개의 게임들 분석하기

sales100 = sorted(df.Total_Sales, reverse=True)
sales100[99] # Total_Sales 상위 100번째 게임의 출고량

In [ ]:
df_100 = df[df.Total_Sales>=734]
df_100

In [ ]:
df_100.groupby('Genre').count() # 장르는 shooter, platform, action이 다수를 차지

In [ ]:
df_100.groupby('Platform_new').count() # platform은 Nintendo 계열이 대부분

In [ ]:
df_100["Game_Series"] = 'a'
for i in range(len(df_100)):
    if 'Gran Turismo' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Gran Turismo'
    elif 'Pok' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Pokemon'
    elif 'Grand Theft Auto' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Grand Theft Auto'
    elif 'Call of Duty' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Call of Duty'
    elif 'Super' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Super Mario'
    elif 'Mario Kart' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Mario Kart'
    elif 'Wii' in df_100.iloc[i, 0]:
        df_100.iloc[i, 11] = 'Wii'
    elif 'Brain' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Brain Age'
    elif 'Halo' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Halo'
    elif 'Final' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Final Fantasy'
    elif 'Just' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Just Dance'
    elif 'FIFA' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'FIFA'
    elif 'Animal' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Animal Crossing'
    elif 'Mario Party' in df_100.iloc[i, 0]:    
        df_100.iloc[i, 11] = 'Mario Party'
    else:
        df_100.iloc[i, 11] = df_100.iloc[i, 0]

In [ ]:
sales = df_100.groupby('Game_Series')['Total_Sales'].sum()

In [ ]:
names = sorted(df_100.Game_Series.unique())

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x=names,
            y=sales)
plt.title('Sales of Series')
plt.xticks(rotation=90);

In [ ]:
# Super Mario, Wii, Pokemon 시리즈의 sales가 상당히 높은 것을 볼 수 있음
# 이 세 시리즈 모두 nintendo를 platform으로 갖기 때문에 이를 확인해보면,
sales_1000 = df_100.groupby('Platform_new')['Total_Sales'].sum()

In [ ]:
platform_1000 = sorted(df_100.Platform_new.unique())

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
plt.title('Market Share of Each Platform', fontsize=14)
pie = ax.pie(x=sales_1000, 
        labeldistance=1.0,
        autopct=lambda p : '{:.2f}%'.format(p))
plt.legend(pie[0],platform_1000) ## 범례 표시
plt.show()

total sales출고량 상위 100개의 게임들은 대부분 유명한 게임 시리즈들이고

nintendo 기기를 사용하는 게임들이 많음

특히 Super Mario, Mario Kart, Mario Party 모두 Mario IP를 활용한 게임이기 때문에

Mario IP의 인기가 상당함을 알 수 있음

In [ ]:
# Mario IP의 지역별 출고량 비율
mario = df_100[df_100.Game_Series.str.contains('Mario')]
print(mario.NA_Sales.sum() / mario.Total_Sales.sum())
print(mario.JP_Sales.sum() / mario.Total_Sales.sum())
# 일본에서 만든 게임이지만 매출이 일본에 국한되지 않음을 알 수 있음

In [ ]:
df.head()

In [ ]:
# df에서 각 지역별 sales만을 사용해 새로운 feature를 추출해보고자 함

df_regions = df[['NA_Sales', 'EU_Sales', 'JP_Sales']]
df_regions.head()
regions = ['NA_Sales', 'EU_Sales', 'JP_Sales']

In [ ]:
from sklearn.preprocessing import StandardScaler # 표준화해주는 method
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
plt.style.use("seaborn")

In [ ]:
scaler = StandardScaler()
Z = scaler.fit_transform(df_regions)
pca = PCA()
pca.fit(df_regions)
B = pca.transform(df_regions)
ratio = pca.explained_variance_ratio_

In [ ]:
def scree_plot(area):
    num_components = len(ratio)
    ind = np.arange(num_components)
    
    ax = plt.subplot()
    cumvals = np.cumsum(ratio)
    ax.bar(ind, ratio) # bar plot
    ax.plot(ind, cumvals, color='#c0392b') # line plot
    
    for i in range(num_components):
        ax.annotate(r"%s"%((str(ratio[i]*100)[:3])), (ind[i], ratio[i]), va="bottom", ha="center", fontsize=13)
        
    ax.set_xlabel("PC")
    ax.set_ylabel("Variance")
    plt.title('Scree plot')
scree_plot(pca)

In [ ]:
def biplot(score,coeff,pcax,pcay,labels=None):
  pca1=pcax-1
  pca2=pcay-1
  xs = score[:,pca1]
  ys = score[:,pca2]
  n=score.shape[1]
  scalex = 1.0/(xs.max()- xs.min())
  scaley = 1.0/(ys.max()- ys.min())
  plt.scatter(xs*scalex,ys*scaley)
  for i in range(n):
    plt.arrow(0, 0, coeff[i,pca1], coeff[i,pca2],color='r',alpha=0.5)
    if labels is None:
      plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, "Var"+str(i+1), color='g', ha='center', va='center')
    else:
      plt.text(coeff[i,pca1]* 1.15, coeff[i,pca2] * 1.15, labels[i], color='g', ha='center', va='center')
  plt.xlim(-1,1)
  plt.ylim(-1,1)
  plt.xlabel("PC{}".format(pcax))
  plt.ylabel("PC{}".format(pcay))
  plt.grid()
biplot(B, pca.components_, 1, 2, labels=regions)

# PC1에 가장 큰 영향을 주고있는 것은 NA_Sales()
# NA_Sales의 분산이 제일 크기 때문에

In [ ]:
df_pca = pd.DataFrame(B, columns=['PC1', 'PC2', 'PC3'])
sns.scatterplot(df_pca.PC1, df_pca.PC2, hue=df.Platform_new)

PCA를 통해 어떤 유의미한 결과를 도출하지 못함

In [ ]:
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_regions)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show();

In [ ]:
from sklearn.preprocessing import Normalizer
nom = Normalizer().fit(df_regions)
nom.transform(df_regions)
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(df_regions)
labels = kmeans.labels_
df['cluster'] = labels
df

In [ ]:
df.cluster.unique()

In [ ]:
X = df.Genre
Y = df.Total_Sales
sns.scatterplot(x=X, y=Y, hue=df.cluster)

In [ ]:
df_regions_100 = df_100[['NA_Sales', 'EU_Sales', 'JP_Sales']]
scaler = StandardScaler()
Z_100 = scaler.fit_transform(df_regions_100)
pca = PCA()
pca.fit(df_regions_100)
B_100 = pca.transform(df_regions_100)
ratio_100 = pca.explained_variance_ratio_

def scree_plot(area):
    num_components = len(ratio_100)
    ind = np.arange(num_components)
    
    ax = plt.subplot()
    cumvals = np.cumsum(ratio_100)
    ax.bar(ind, ratio_100) # bar plot
    ax.plot(ind, cumvals, color='#c0392b') # line plot
    
    for i in range(num_components):
        ax.annotate(r"%s"%((str(ratio_100[i]*100)[:3])), (ind[i], ratio_100[i]), va="bottom", ha="center", fontsize=13)
        
    ax.set_xlabel("PC")
    ax.set_ylabel("Variance")
    plt.title('Scree plot')
scree_plot(pca)

In [ ]:
df_100_pca = pd.DataFrame(B, columns=['PC1', 'PC2', 'PC3'])
sns.scatterplot(df_100_pca.PC1, df_100_pca.PC2, hue=df_100.Year)

PCA를 진행해보았지만 특이점 찾지 못함

In [ ]:
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_regions_100)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show();

In [ ]:
from sklearn.preprocessing import Normalizer
nom = Normalizer().fit(df_regions)
nom.transform(df_regions_100)
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(df_regions_100)
labels = kmeans.labels_
df_100['cluster'] = labels
df_100

In [ ]:
X = df_100.NA_Sales
Y = df_100.EU_Sales
sns.scatterplot(x=X, y=Y, hue=df_100.cluster)

In [ ]:
# cluster 1로 나누어진 data
df_100[df_100.cluster==1]

In [ ]:
# cluster 2로 나누어진 data
df_100[df_100.cluster==2]

clustering 진행해보았지만 유의미한 결과를 도출하지 못함

NA_Sales와 EU_Sales가 모두 높은 data, NA_Sales만 높은 data, NA_Sales와 EU_Sales가 중간인 data, 모두 낮은 data 이렇게 cluster가 나누어진듯함

In [ ]:
# 시대별 각 지역별 판매량이 어떻게 변화하는지 알아보고자 함
plt.figure(figsize=(8,6))
sns.set_theme(color_codes=True)


sns.regplot(x="Year", y="NA_Sales", data=df_100, label="NA")
sns.regplot(x="Year", y="EU_Sales", data=df_100, label="EU")
sns.regplot(x="Year", y="JP_Sales", data=df_100, label="JP")
# plt.ylim(0, 2000)
plt.xlabel('Year')
plt.ylabel('Sales')
plt.title('Sales by Year')
plt.legend()

연도가 증가함에 따라 NA와 JP 지역에서의 판매량은 줄어들고 있음

하지만 이건 실제로 판매량이 줄어드는 것이 아니라 다양한 게임이 나오기 때문에
상대적으로 각 게임의 판매량이 줄어드는 현상이었음

또 EU 지역은 NA와 JP랑은 다르게 다양한 게임이 나옴에도 불구하고 전반적인 게임의 판매량이 증가함을 보임

이는 과거에는 EU 지역에서의 게임 산업이 어려웠지만 점차 게임에 대한 관심도가 증가한다고 볼 수 있음

In [ ]:
data_NA = [year_1990.NA_Sales.sum(), year_2000.NA_Sales.sum(), year_2010.NA_Sales.sum(), year_2020.NA_Sales.sum()]
data_EU = [year_1990.EU_Sales.sum(), year_2000.EU_Sales.sum(), year_2010.EU_Sales.sum(), year_2020.EU_Sales.sum()]
data_JP = [year_1990.JP_Sales.sum(), year_2000.JP_Sales.sum(), year_2010.JP_Sales.sum(), year_2020.EU_Sales.sum()]
years = ['1980s', '1990s', '2000s', '2010s']

country_NA=[1,3,5,7]
country_EU=[1+0.5,3+0.5,5+0.5,7+0.5]
country_JP=[1+1,3+1,5+1,7+1]

plt.figure(figsize=(8,6))
plt.bar(country_NA, data_NA, label="NA", color='r',width=0.5)
plt.bar(country_EU, data_EU, label="EU", color='g',width=0.5)
plt.bar(country_JP, data_JP, label="JP", color='b',width=0.5)
ticklabel=['1980s','1990s','2000s','2010s']
plt.xlabel('Year')
plt.ylabel('Sales')
plt.title('Sales by Year')
plt.xticks(country_EU,ticklabel,fontsize=15,rotation=0)
plt.legend();

JP지역은 꾸준하게 우상향하는 모습을 보이고, 1980년대까진 NA지역이 게임시장 매출의 대부분을 차지하였지만 EU지역도 크게 성장하였음

하나의 타이틀이 여러 플랫폼에서 출시될 수도 있다

그렇다면 어떤 장르가 어떤 플랫폼에서 유리할까?

In [ ]:
# 분석에 필요하지 않은 columns 삭제하고 새로운 변수명 df2에 저장

df2 = df.drop(['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'cluster'], axis=1)
df2

In [ ]:
# 같은 title로 여러 개의 platform에 출시한 게임들만 빼서 df_multi로 저장
# 더불어 2010년대 이후에 출시한 게임들만 필터링
df_multi = df2[(df2.Name.duplicated(keep=False)) & (df2.Year >= 2010)]
df_multi

In [ ]:
# 각 장르별 platform에 따른 출고량 비교해보기

genres = df_multi.Genre.unique()
fig, axes = plt.subplots(nrows=2, ncols=6, figsize=(40, 20))
for row in range(2):
    for col in range(6):
        idx = row*6+col

        if idx < len(genres):
            ax = axes[row][col]
            genre = genres[idx]
            sns.barplot(x=df_multi[df_multi.Genre==genre].Platform_new,
                y=df_multi[df_multi.Genre==genre].Total_Sales, ax=ax)
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=10)
            ax.set(xlabel=genre)

위 내용들을 종합했을 때 만약 우리 주력 상품이 Action 장르의 게임이라면 playstation과 Xbox의 multiplatform으로 만들고 action 장르가 강세인 NA와 EU 지역을 중점적으로 타겟해야한다